In [36]:
from selenium import webdriver
import json
from bs4 import BeautifulSoup


# Set up the browser (Chrome in headless mode)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=options)

url = "https://leetcode.com/studyplan/top-100-liked/"

driver.get(url)

html = driver.page_source

rs = BeautifulSoup(html, "html.parser")

content = rs.find('script', id='__NEXT_DATA__').text


data = json.loads(content)

plan_details = {}

main = data['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']['studyPlanV2Detail']

plan_details['slug'] = main['slug']
plan_details['name'] = main['name']
plan_details['problems'] = []

for section in main['planSubGroups']:
    s = {}
    s['name'] = section['name']
    s['size'] = section['questionNum']
    s['problems'] = []
    for problem in section['questions']:
        p = {}
        p['slug'] = problem['titleSlug']
        p['name'] = problem['title']
        p['difficulty'] = problem['difficulty']
        p['url'] = f"https://leetcode.com/problems/{problem['titleSlug']}"
        p['tags'] = [tag['name'] for tag in problem['topicTags']]
        s['problems'].append(p)
    plan_details['problems'].append(s)

with open( f'./study-plans/{plan_details['slug']}.json', 'w') as f:
    json.dump(plan_details, f, indent=4)




In [22]:
import json
import os
from dotenv import load_dotenv
import psycopg2

# Load variables from .env file
load_dotenv('../backend/.env')

# Access them using os.environ
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASSWORD")


# Replace with your actual values
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_pass
)

cur = conn.cursor()

files = os.listdir('new-plans')
for file_path in files:
    with open(f'new-plans/{file_path}', 'r') as file:
        data = json.load(file)

    try:
        plan_name = data['name']
        plan_slug = data['slug']
        plan_source = data['source']
        plan_source_link = data['source_link']

        # create a new plan
        print(plan_name,plan_slug,plan_source,plan_source_link)
        cur.execute("INSERT INTO plan (name, slug, source, source_link) VALUES (%s, %s, %s, %s) RETURNING plan_id", (plan_name, plan_slug, plan_source, plan_source_link))
        plan_id = cur.fetchone()[0]
        conn.commit()
        print(f"Plan created with id: {plan_id}")

        for section_order,section in enumerate(data['problems']):
            section_name = section['name']
            section_problems = section['problems']
            for problem in section_problems:
                problem_name = problem['name']
                problem_slug = problem['slug']
                problem_difficulty = (problem['difficulty']=='EASY')*1 + (problem['difficulty']=='MEDIUM')*2 + (problem['difficulty']=='HARD')*3
                problem_url = problem['url']
                problem_tags = problem['tags']
                cur.execute("INSERT INTO problem (name, slug, difficulty, url, tags, plan_id,topic,topic_order) VALUES (%s, %s, %s, %s, %s, %s,%s,%s)", (problem_name, problem_slug, problem_difficulty, problem_url, problem_tags, plan_id,section_name,section_order))
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
        continue

conn.close()




Top Interview 150 top-interview-150 leetcode https://leetcode.com/studyplan/top-interview-150/
Plan created with id: 1
Graph Theory graph-theory leetcode https://leetcode.com/studyplan/graph-theory/
Plan created with id: 2
LeetCode 75 leetcode-75 leetcode https://leetcode.com/study-plan/leetcode-75/
Plan created with id: 3
Top SQL 50 top-sql-50 leetcode https://leetcode.com/studyplan/top-sql-50/
Plan created with id: 4
Programming Skills programming-skills leetcode https://leetcode.com/studyplan/programming-skills/
Plan created with id: 5
Top 100 Liked top-100-liked leetcode https://leetcode.com/studyplan/top-100-liked/
Plan created with id: 6
Binary Search binary-search leetcode https://leetcode.com/studyplan/binary-search/
Plan created with id: 7
Dynamic Programming dynamic-programming leetcode https://leetcode.com/studyplan/dynamic-programming/
Plan created with id: 8
Introduction to Pandas introduction-to-pandas leetcode https://leetcode.com/studyplan/introduction-to-pandas/
Plan c

In [20]:
cur.execute("SELECT * FROM plan")
rows = cur.fetchall()
rows


[]